In [2]:
# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
import pandas as pd
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nyra_2019_complete = '1_yb_57qPKmYDx9gI3GyEeIh6FuLWvejL'
modelo = '1RRkNN12t8yaCV7dI2rBSf5NM5wjhqpjy' #<-- Key del file en drive (nyra_2019_complete.csv 610 mb)
# nyra_race_table    = '1KtYQEvt4pboSqs7bO6jfdCHADo9PMzo9'#<-- Key del file en drive (nyra_race_table.csv 93 kb)
# nyra_start_table   = '1LYUx_De3U7TBw6GgqRk6whVzMjuJjSaz'#<-- Key del file en drive (nyra_start_table.csv 722 Kb)
# nyra_tracking_table= '15y4AtfLiyop9FWdc0SRaMUs-jKgnOppl'#<-- Key del file en drive (nyra_tracking_table.csv 320,7 mb)


download = drive.CreateFile({'id': nyra_2019_complete}) 
download1 = drive.CreateFile({'id': modelo}) 
# Download the file to a local disc
download.GetContentFile('carrera.csv')
download1.GetContentFile('model.h5')
carrera = pd.read_csv('carrera.csv')

from tensorflow.keras.models import load_model
model = load_model('model.h5')

#Calcula posiciones de odds
positions = pd.DataFrame(
    carrera
    .groupby(["race_number", "race_date"])["track_id"]
    .count()
).reset_index()
positions = positions.rename(columns={"track_id": "participantes"})

carrera["odds_ranking"] = carrera.groupby(["race_number", "race_date"])[
    "odds"
].rank(method="first", ascending=False)
carrera.sort_values(by=["race_date", "track_id", "race_number"])

carrera = carrera.merge(
    positions, on=["race_number", "race_date"], how="left"
)
carreras_test = carrera[
    [
        "race_date",
        "race_number",
        "position_at_finish",
        "First_3",
        "weight_carried",
        "odds_ranking",
        "track_id",
        "Percentage_first_3"
    ]
]


Predictors = ["weight_carried", "odds_ranking", "Percentage_first_3"]
X_test = carreras_test[Predictors]

dataframe = pd.DataFrame(model.predict(X_test).tolist())
dataframe = dataframe.rename(columns={0: "prediccion"})

Base_prediccion = pd.concat([carreras_test, dataframe], axis=1).sort_values(
    by=["race_date", "track_id", "race_number", "odds_ranking", "position_at_finish"]
)
Base_prediccion["Prediccion_ranking"] = Base_prediccion.groupby(
    ["race_number", "race_date"]
)["prediccion"].rank(method="first", ascending=False)
Base_prediccion["First_3_ranking"] = np.where(
    Base_prediccion["Prediccion_ranking"].between(0, 5, inclusive=False),
    1,
    np.where(
        Base_prediccion["Prediccion_ranking"].between(4, 30, inclusive=False),
        0,
        "Unknown",
    ),
)

from sklearn import metrics

print(
    "Precision:",
    metrics.precision_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "Recall:",
    metrics.recall_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "Accuracy:",
    metrics.accuracy_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
print(
    "F1_Score:",
    metrics.f1_score(
        Base_prediccion[["First_3"]].astype(int),
        Base_prediccion[["First_3_ranking"]].astype(int),
    ),
)
#Resultados = Resultados.append(Resultados1, ignore_index=True)

Base_prediccion

1/1 [==============================] - 0s 442ms/step
Precision: 0.5
Recall: 0.6666666666666666
Accuracy: 0.5
F1_Score: 0.5714285714285715


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.


,race_date,race_number,position_at_finish,First_3,weight_carried,odds_ranking,track_id,Percentage_first_3,prediccion,Prediccion_ranking,First_3_ranking
0,2019-01-01,4,5,0,122,1.0,AQU,0.5,0.209166,6.0,0
1,2019-01-01,4,2,1,122,2.0,AQU,0.5,0.302238,5.0,0
2,2019-01-01,4,6,0,122,3.0,AQU,0.5,0.399360,4.0,1
3,2019-01-01,4,3,1,124,4.0,AQU,0.5,0.469480,3.0,1
4,2019-01-01,4,4,0,124,5.0,AQU,0.5,0.524143,2.0,1
5,2019-01-01,4,1,1,118,6.0,AQU,0.5,0.570543,1.0,1
